### Лекция 7 - ООП в Python
- особенности инкапсуляции, наследования и полиморфизма в Python
- метаклассы
- динамическое создание классов
- MRO
- пользовательские исключения
- менеджеры контекста
- глубокое копирование
- декораторы через классы
- класс enum
- модуль attrs

<center>Что хотим сделать</center>
```
interface IParser
{
	void Load();
	string Parse();	
}

class XMLParser : IParser
{
	void Load() { ... }
	string Parse() { ... }
}

class JSONParser : IParser
{
	void Load() { ... }
	string Parse() { ... }
}
```

In [8]:
# ==========================================================
#              ПОЛЬЗОВАТЕЛЬСКИЕ ИСКЛЮЧЕНИЯ
# ==========================================================

class FileTooBigError(Exception):

    def __init__(self, filesize):
        super(FileTooBigError, self).__init__(
            'Filesize is too big: {} bytes'.format(filesize))
        self.filesize = filesize

    # если __str__() должен возвращать просто сообщение message, 
    # то метод можно не определять 
    # (по умолчанию, строковое представление Exception - и так message)
    # def __str__(self):
    #    return self.message


import os

fsize = os.path.getsize(r'lec01 - Intro.ipynb')

try:
    if fsize > 4096:
        raise FileTooBigError(fsize)
except FileTooBigError as err:
    print(err)

Filesize is too big: 34227 bytes


In [13]:
# ==========================================================
#                      МЕНЕДЖЕРЫ КОНТЕКСТА
# ==========================================================
# ПЕРВЫЙ СПОСОБ:

class PingContext(object):

    def __enter__(self):
        self.log = open(r'data\log.txt', 'wt')
        print('Log file is ready')
        return self

    def write(self, text):
        self.log.write(text)
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        if exc_value != None:
            print('Ping failed! Wrong IP format: %s' % exc_value)
            # подчистить ресурсы, которыми владеет контекст
        self.log.close()
        print('Log file is closed')    
        return True
        #return False          # если False, то заново бросается Exception
                               # (и надо будет перехватывать во внешнем коде)


IPs = ['127.0.0.1', '0.0.0.0', '1.1.1.1', '10.0.0.1', 'f.2.3.4', '192.168.0.102']

# регулярка для простейшей проверки валидности IP-адреса
import re
regex = re.compile(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')

with PingContext() as log:
    # проходим по всем записям из строк IP-адресов
    for IP in IPs:
        if not re.match(regex, IP):
            raise ValueError(IP)
        # пинг IP-адреса ...
        # и запись в лог
        log.write(IP + '\n')

Log file is ready
Ping failed! Wrong IP format: f.2.3.4
Log file is closed


In [3]:
# === ВТОРОЙ СПОСОБ:

from contextlib import contextmanager

@contextmanager
def ping_log():
    # __enter__
    print('Log file is ready')
    log = open(r'data\log.txt', 'wt')

    # actions
    try:
        yield log

    # __exit__    
    except ValueError as err:
        print('Ping failed! Wrong IP format: %s' % err)
    finally:
        log.close()
        print('Log file is closed')


regex = re.compile(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')

IPs = ['127.0.0.1', '0.0.0.0', '1.1.1.1', '10.0.0.1', 'f.2.3.4', '192.168.0.102']

with ping_log() as log:
    for IP in IPs:
        if not re.match(regex, IP):
            raise ValueError(IP)
        log.write(IP + '\n')

Log file is ready
Ping failed! Wrong IP format: f.2.3.4
Log file is closed


In [1]:
# демо глубокого копирования стандартных типов
import copy

l1 = [1, "Hello", [3,4,5], (2, [7,8,9], 'x'), 12]

l2 = copy.deepcopy(l1)
l2[1] = "World"

print(l1)
print(l2)

[1, 'Hello', [3, 4, 5], (2, [7, 8, 9], 'x'), 12]
[1, 'World', [3, 4, 5], (2, [7, 8, 9], 'x'), 12]


In [9]:
# демо глубокого копирования объектов классов
from fractions import Fraction

f1 = Fraction(3, 5)
f2 = copy.deepcopy(f1)  # дефолтная реализация;
                        # ее можно переопределить в классе Fraction: 
                        # метод __deepcopy__(self, f)
print(f1)
print(f2)

3/5
3/5


In [10]:
dir(f1)

['__abs__',
 '__abstractmethods__',
 '__add__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__complex__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__round__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_add',
 '_denominator',
 '_div',
 '_mul',
 '_numerator',
 '_operator_fallbacks',
 '_richcmp',
 '_sub',
 'conjugate',
 'denominator',
 'from_decimal',
 'from_float',